In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
N = 100000
np.random.seed(1)
data0 = np.random.normal(0, 1.25, N)
data1 = np.random.normal(0, 1, int(N/100))

In [3]:
def train_generator(numsamples=1000):
    n0 = int(0.7*len(data0))
    n1= int(0.7*len(data1))
    if np.random.binomial(1, 0.5):
        # return class 0
        i = np.random.randint(0, n0-60)
        yield data0[i:i+60], 0
    else:
        # return class 1
        i = np.random.randint(0, n1-60)
        yield data1[i:i+60], 1
        
def test_generator():
    d0 = data0[-int(0.3*len(data0)):]
    d1 = data1[-int(0.3*len(data1)):]
    for i in range(len(d0)//60-1):
        yield d0[60*i:60*(i+1)], 0
    for i in range(len(d1)//60-1):
        yield d1[60*i:60*(i+1)], 1
        
def train_input_fn():
    dataset = tf.data.Dataset.from_generator(train_generator, (tf.float32, tf.int32),
                                            ((60), ())).batch(100).prefetch(10)
    return dataset.make_one_shot_iterator().get_next()

def test_input_fn():
    dataset = tf.data.Dataset.from_generator(test_generator, (tf.float32, tf.int32),
                                            ((60), ())).batch(100).prefetch(10)
    return dataset.make_one_shot_iterator().get_next()

In [4]:
def model_fn(features, labels, mode, params):
    """
    """
    labels_oh = tf.one_hot(labels, 2)
    # 60
    net = tf.expand_dims(features, 2)
    net = tf.layers.conv1d(net, 8, 3, activation=tf.nn.relu)
    net = tf.layers.max_pooling1d(net, 2, 2)
    # 30
    net = tf.layers.conv1d(net, 16, 3, activation=tf.nn.relu)
    net = tf.layers.max_pooling1d(net, 2, 2)
    # 15
    net = tf.layers.conv1d(net, 32, 3, activation=tf.nn.relu)
    net = tf.layers.max_pooling1d(net, 3, 3)
    # 5
    net = tf.layers.conv1d(net, 64, 3, activation=tf.nn.relu)
    flat = tf.layers.flatten(net)
    logits = tf.layers.dense(flat, 2)
    predicted_class = tf.argmax(logits, 1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions={"class_ids":predicted_class[:, tf.newaxis],
                    "probabilities":tf.nn.softmax(logits),
                    "logits":logits}
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    loss = tf.losses.softmax_cross_entropy(labels_oh, logits)
    # compute eval metrics
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predicted_class)
    metrics={"accuracy":accuracy}
    tf.summary.scalar("accuracy", accuracy[1])
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics
        )
    optimizer = tf.train.MomentumOptimizer(0.01, 0.9)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [5]:
model = tf.estimator.Estimator(model_fn=model_fn, params={}, model_dir="logs")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'logs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x181d99abe0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
trainspec = tf.estimator.TrainSpec(train_input_fn)

In [7]:
evalspec = tf.estimator.EvalSpec(test_input_fn)

In [9]:
tf.estimator.train_and_evaluate?

Signature: tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
Docstring:
Train and evaluate the `estimator`.

This utility function trains, evaluates, and (optionally) exports the model by
using the given `estimator`. All training related specification is held in
`train_spec`, including training `input_fn` and training max steps, etc. All
evaluation and export related specification is held in `eval_spec`, including
evaluation `input_fn`, steps, etc.

This utility function provides consistent behavior for both local
(non-distributed) and distributed configurations. Currently, the only
supported distributed training configuration is between-graph replication.

Overfitting: In order to avoid overfitting, it is recommended to set up the
training `input_fn` to shuffle the training data properly. It is also
recommended to train the model a little longer, say multiple epochs, before
performing evaluation, as the input pipeline starts from scratch for each
training. It is partic

In [8]:
tf.estimator.train_and_evaluate(model, trainspec, evalspec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into logs/model.ckpt.
INFO:tensorflow:loss = 0.7347924, step = 1
INFO:tensorflow:Loss for final step: 0.7347924.
INFO:tensorflow:Starting evaluation at 2018-05-16-01:07:57
INFO:tensorflow:Restoring parameters from logs/model.ckpt-1
INFO:tensorflow:Finished evaluation at 2018-05-16-01:07:57
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.6023857, global_step = 1, loss = 0.68919784
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from logs/model.ckpt-1
INFO:tensorflo

KeyboardInterrupt: 